In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [7]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [3]:
essence_url = []
for i in range (1, 44):
    url = f'https://www.yesstyle.com/en/women-face-serums-ampoules/list.html/bcc.14339_bpt.46#/bt=37&sb=136&bpt=46&pn={i}&s=10&bcc=14339&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    essence_list = soup.find_all('div', class_='itemContainer')
    for item in essence_list:
        for link in item.find_all('a', href=True): 
            essence_url.append(link['href'])


In [4]:
print(len(essence_url))

1548


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [51]:
"""
test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text
"""

In [6]:
essence_url[0]

'https://www.yesstyle.com/en/the-face-shop-real-nature-face-mask-1pc-20-types-20g-calendula/info.html/pid.1061020764'

In [5]:
essence = []

In [8]:
for link in essence_url[0:1548]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text.strip() #ingredients
        rating = soup.find('div', class_='satisfied').text.strip()
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    essence_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    essence.append(essence_info)

In [9]:
print(len(essence))

1548


In [12]:
essence[496]

IndexError: list index out of range

## 3. PASARLO A DATAFRAME

In [10]:
essence_df = pd.DataFrame(essence)
essence_df

,name,rating,ingredients
0,\nCOSRX - Advanced Snail 96 Mucin Power Essence\n,"97.7% customers satisfied|4,210 reviews","Snail Secretion Filtrate, Betaine, Butylene Gl..."
1,"\nSOME BY MI - AHA, BHA, PHA 30 Days Miracle S...","93.3% customers satisfied|2,374 reviews","Water, Centella Asiatica (Gotu Kola) Extract (..."
2,\nBenton - Snail Bee High Content Essence 60ml\n,"96.5% customers satisfied|4,185 reviews","Aqua (Water), Snail Secretion Filtrate, Butyle..."
3,\nBeauty of Joseon - Repair Serum\n,98.1% customers satisfied|407 reviews,"Panax Ginseng Root Water, Butylene Glycol, 1,2..."
4,\nRohto Mentholatum - Melano CC Vitamin C Esse...,"96.1% customers satisfied|1,842 reviews",Active ingredient : Ascorbic acid (Active vita...
...,...,...,...
1543,\nSOME BY MI - Yuja Niacin 30 Days Blemish Car...,92.9% customers satisfied|286 reviews,"Citrus Junos Fruit Extract (82.37%), Niacinami..."
1544,\nTimeless Skin Care - 20% Vitamin C + E Ferul...,91.9% customers satisfied|303 reviews,"Water, Ethoxydiglycol, L-Ascorbic Acid, Propyl..."
1545,\nSecret Key - Multi Cell Night Repair Ampoule...,93.4% customers satisfied|432 reviews,"Water, Glycerin, Butylene Glycol, Arbutin, Nia..."
1546,\nIsntree - TW-Real Bifida Ampoule 50ml\n,99% customers satisfied|103 reviews,"Bifida Fermentation Lysate, Dipropylene Glycol..."


In [11]:
essence = essence_df.to_csv('yesstyle_data/essence.csv')